# 1. Introduction

In my last notebook on [Natural Language Processing (NLP)](https://www.kaggle.com/gargimaheshwari/nlp-machine-learning-topics-part-a) I used <code>sklearn</code>'s GridSearch to find the best LDA model for the number of macine learning topics in the NIPS papers. We saw that using <code>search_params</code> over <code>[20, 25, 30, 35]</code> number of topics and <code>[10, 20, 50 ]</code> maximum iterations, the best model turns out to be the one with 20 topics and 50 iterations, with a perplexity of approximately 2453.

Here, I would like to take that analysis further.

An important point to note is that perplexity alone might not be the best measure to evaluate topic models because it doesn’t consider the context and semantic associations between words. These can be captured using *topic coherence measure*. I will explore this measure here.

First, I'll load the necessary libraries and the data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re

papers = pd.read_csv('../input/papers.csv')
papers.head()

As before, I will process the database to remove unnecessary fields, and then process the text using the <code>re</code> library.

In [ ]:
papers = papers.drop(["id", "event_type", "pdf_name"], axis = 1)

papers['title_processed'] = papers['title'].map(lambda x: re.sub('[,\.!?]', '', x))
papers['title_processed'] = papers['title_processed'].map(str.lower)

# 2. Pre-processing 

To further process the data and to ready it for LDA to consume, it needs to be tokenized. However, unlike last time, I will not use <code>sklearn</code>'s resources, but instead I will use NLTK and Gensim. This is because our further analysis of topic coherence uses Gensim's resources, and we need to maintain continuity in the API.

In [ ]:
data = [title.split() for title in papers['title_processed']]

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data = remove_stopwords(data)
id2word = corpora.Dictionary(data)
corpus = [id2word.doc2bow(text) for text in data]

# 3. LDA Topic Coherence measure

An approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value. Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics. If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The function below trains multiple LDA models and provides the models and their corresponding coherence scores.

In [ ]:
from gensim.models import CoherenceModel

def compute_coherence_values(dictionary, data, corpus, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = dictionary,
                                           num_topics = num_topics,
                                           random_state = 10)
        model_list.append(model)
        coherencemodel = CoherenceModel(model = model,
                                        texts = data, 
                                        dictionary = dictionary,
                                        coherence = 'c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

Since the last time around we got the ideal number of topics as 20, I will search between 10 and 30 for the ideal number of topics and run the function above. This can take a while to run as the function trains multiple LDA models, however it is not as slow as the GridSearch before.

In [ ]:
start = 10; limit = 30; step = 2

model_list, coherence_values = compute_coherence_values(dictionary = id2word, 
                                                        data = data, corpus = corpus, 
                                                        start = start, limit = limit,
                                                        step = step)

I will now plot the coherence values, and choose the optimal model based on the figure.

In [ ]:
x = range(start, limit, step)
plt.figure(figsize = (13, 4))
plt.plot(x, coherence_values, color = 'indigo')
plt.xlabel("Num Topics", fontsize = 13)
plt.ylabel("Coherence score", fontsize = 13)
plt.xticks(x)
plt.show()

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 3))

If the coherence score seems to keep increasing, it makes sense to pick the model that gave the highest CV before flattening out. This is exactly the case here.

So for further steps I will choose the model with 20 topics itself and print out the three most dominant words in each topic.

In [ ]:
optimal_model = model_list[5]
model_topics = optimal_model.show_topics(20, formatted = False)
for i in range(20):
    print((model_topics[i][0] + 1), (list(model_topics[i][1][j][0] for j in range(3))))

Finally, as a last step, I will use python's <code>sklearn</code> to create an LDA model with 20 topics, and use the LDA visualizer to view the topics-keywords distribution. A good topic model will have non-overlapping, fairly big sized blobs for each topic. This seems to be the case here. So, we are good.

The visalization is also available in the output tab on the left, under <code>vis.html</code>.

In [ ]:
import warnings
warnings.simplefilter("ignore", FutureWarning)

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words = 'english')
count_data = count_vectorizer.fit_transform(papers['title_processed'])

from sklearn.decomposition import LatentDirichletAllocation as LDA
lda = LDA(n_components = 20, random_state = 10)
model = lda.fit(count_data)

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(model, count_data, count_vectorizer, mds='tsne')
saved = pyLDAvis.save_html(vis, fileobj = "vis.html")

![](https://i.imgur.com/H87fvR3.jpg)